In [ ]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.1
    Uninstalling pip-23.3.1:
      Successfully uninstalled pip-23.3.1
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
pathos 0.3.1 requires dill>=0.3.7, but you have dill 0.3.6 which is incompatible.
pathos 0.3.1 requires multiprocess>=0.70.15, but you have multiprocess 0.70.14 which is incompatible.
sagemaker 2.199.0 requires urllib3<1.27, but you have urllib3 2.1.0 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.18.1 which is incompatible.
spyder 5.3.3 requires pylint<3.0,>=2.5.0, but you have pylint 3.0.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
})

In [4]:
%%time
# Import SageMaker JumpStartModel package
from sagemaker.jumpstart.model import JumpStartModel

# Define model_id and model_version
model_id, model_version, = (
    "huggingface-llm-falcon-7b-bf16",
    "*"
)

# deploy the model for prediction
my_model = JumpStartModel(model_id=model_id, instance_type="ml.g5.4xlarge")
predictor = my_model.deploy()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


Using model 'huggingface-llm-falcon-7b-bf16' with wildcard version identifier '*'. You can pin to version '2.0.1' for more stable results. Note that models may have different input/output signatures after a major version upgrade.


----------!CPU times: user 860 ms, sys: 106 ms, total: 966 ms
Wall time: 5min 34s


In [5]:
%%time

human_baseline_summaries = []
original_model_summaries = []

# Zero shot prompting for dataset index 200 
index = 200

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

human_baseline_summaries.append(dataset['test'][0:10]['summary'])


# Prompt preparation
prompt = f"""
Summarize the following conversation

{dialogue}

Summary:

"""
print(prompt)

payload = {
    "inputs": prompt,
    "parameters": {
        "do_sample": False, 
        "top_p": 0.9,
        "temperature": 0.8,
        "max_new_tokens": 1024,
        "stop": ["<|endoftext|>", "<\/s>" ],
    },
}

# predict the response
response = predictor.predict(payload)
print(response[0]["generated_text"])
original_model_text_output = response[0]["generated_text"]
original_model_summaries.append(original_model_text_output)


Summarize the following conversation

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary:


#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your sof

In [6]:
# print input prompt, human summary, and model generated - Zero shot
dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{original_model_text_output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary:


-------------------------------------------------------------------

In [7]:
# print evaluate package to create rouge
import evaluate
rouge = evaluate.load('rouge')

In [8]:
# Evaluate rouge metrics for Zero shot

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)


print('ORIGINAL MODEL:')
print(original_model_results)

ORIGINAL MODEL:
{'rouge1': 0.04594594594594594, 'rouge2': 0.0027739251040221915, 'rougeL': 0.03513513513513514, 'rougeLsum': 0.03513513513513514}


In [9]:
prompt = f"""
            Summarize the following conversation.

            #Person1#: Have you considered upgrading your system?
            #Person2#: Yes, but I'm not sure what exactly I would need.
            #Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
            #Person2#: That would be a definite bonus.
            #Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
            #Person2#: How can we do that?
            #Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
            #Person2#: No.
            #Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
            #Person2#: That sounds great. Thanks.

            Summary:

            #Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
            
            #############################
            Summarize the following conversation.

            {dialogue}

            Summary: """